In [1]:
from typing import Text
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader

loader = TextLoader("./files/chapter_one.txt", encoding='utf-8')

loader.load()



[Document(page_content='The Project Gutenberg eBook of Pride and Prejudice\n    \nThis ebook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this ebook or online\nat www.gutenberg.org. If you are not located in the United States,\nyou will have to check the laws of the country where you are located\nbefore using this eBook.\n\nTitle: Pride and Prejudice\n\n\nAuthor: Jane Austen\n\nRelease date: June 1, 1998 [eBook #1342]\n                Most recently updated: April 14, 2023\n\nLanguage: English\n\nCredits: Chuck Greif and the Online Distributed Proofreading Team at http://www.pgdp.net (This file was produced from images available at The Internet Archive)\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE ***\n\n\n\n\n                            [Illustration:\n\n      

In [2]:
from langchain.document_loaders import PyPDFLoader

loader2 = PyPDFLoader("./files/chapter_one.pdf")

loader2.load()

[Document(page_content='The Project Gutenberg eBook of Pride and Prejudice  \n     \nThis ebook is for the use of anyone anywhere in the United States and  \nmost other parts of the world at no cost and with almost no restrictions  \nwhatsoever. You may copy it, give it away or re -use it under the terms  \nof the Project Gutenberg License included with this ebook or online  \nat www.gutenberg.org. If you are not located in the United States,  \nyou will have to check the laws of the country where you are located  \nbefore using this eBook.  \n \nTitle: Pride and Prejudice  \n \n \nAuthor: Jane Austen  \n \nRelease date: June 1, 1998 [eBook #1342]  \n                Most recently updated: April 14, 2023  \n \nLanguage: English  \n \nCredits: Chuck Greif and the Online Distributed Proofreading Team at \nhttp://www.pgdp.net (This file was produced from images available at The \nInternet Archive)  \n \n \n*** START OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE ***  \n \n \n \n \n    

In [3]:
from langchain.document_loaders import UnstructuredFileLoader

loader3 = UnstructuredFileLoader("./files/chapter_one.pdf")

loader3.load()

[Document(page_content='The Project Gutenberg eBook of Pride and Prejudice\n\nThis ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook.\n\nTitle: Pride and Prejudice\n\nAuthor: Jane Austen\n\nRelease date: June 1, 1998 [eBook #1342] Most recently updated: April 14, 2023\n\nLanguage: English\n\nCredits: Chuck Greif and the Online Distributed Proofreading Team at http://www.pgdp.net (This file was produced from images available at The Internet Archive)\n\n*** START OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE ***\n\n[Illustration:\n\nGEORGE ALLEN PUBLISHER\n\n156 CHARING CROSS ROAD LONDON\n\nRUSKIN HOUSE

In [4]:
loader4 = UnstructuredFileLoader("./files/chapter_one.docx")

loader4.load()

[Document(page_content='The Project Gutenberg eBook of Pride and Prejudice\n\nThis ebook is for the use of anyone anywhere in the United States and\n\nmost other parts of the world at no cost and with almost no restrictions\n\nwhatsoever. You may copy it, give it away or re-use it under the terms\n\nof the Project Gutenberg License included with this ebook or online\n\nat www.gutenberg.org. If you are not located in the United States,\n\nyou will have to check the laws of the country where you are located\n\nbefore using this eBook.\n\nTitle: Pride and Prejudice\n\nAuthor: Jane Austen\n\nRelease date: June 1, 1998 [eBook #1342]\n\n                Most recently updated: April 14, 2023\n\nLanguage: English\n\nCredits: Chuck Greif and the Online Distributed Proofreading Team at http://www.pgdp.net (This file was produced from images available at The Internet Archive)\n\n*** START OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE ***\n\n                            [Illustration:\n\n      

In [5]:
len(loader4.load())

1

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)

loader4 = UnstructuredFileLoader("./files/chapter_one.docx")

loader4.load_and_split(text_splitter=splitter)


[Document(page_content='The Project Gutenberg eBook of Pride and Prejudice\n\nThis ebook is for the use of anyone anywhere in the United States and\n\nmost other parts of the world at no cost and with almost no restrictions', metadata={'source': './files/chapter_one.docx'}),
 Document(page_content='whatsoever. You may copy it, give it away or re-use it under the terms\n\nof the Project Gutenberg License included with this ebook or online', metadata={'source': './files/chapter_one.docx'}),
 Document(page_content='at www.gutenberg.org. If you are not located in the United States,\n\nyou will have to check the laws of the country where you are located\n\nbefore using this eBook.\n\nTitle: Pride and Prejudice', metadata={'source': './files/chapter_one.docx'}),
 Document(page_content='Title: Pride and Prejudice\n\nAuthor: Jane Austen\n\nRelease date: June 1, 1998 [eBook #1342]\n\n                Most recently updated: April 14, 2023\n\nLanguage: English', metadata={'source': './files/chapte

In [7]:
len(loader4.load_and_split(text_splitter=splitter))

106

In [8]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader5 = UnstructuredFileLoader("./files/chapter_one.docx")

loader5.load_and_split(text_splitter=splitter)

[Document(page_content='The Project Gutenberg eBook of Pride and Prejudice\nThis ebook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this ebook or online\nat www.gutenberg.org. If you are not located in the United States,\nyou will have to check the laws of the country where you are located\nbefore using this eBook.\nTitle: Pride and Prejudice\nAuthor: Jane Austen\nRelease date: June 1, 1998 [eBook #1342]', metadata={'source': './files/chapter_one.docx'}),
 Document(page_content='Title: Pride and Prejudice\nAuthor: Jane Austen\nRelease date: June 1, 1998 [eBook #1342]\n                Most recently updated: April 14, 2023\nLanguage: English\nCredits: Chuck Greif and the Online Distributed Proofreading Team at http://www.pgdp.net (This file was produced from images available at The I

In [9]:
len(loader5.load_and_split(text_splitter=splitter))

31